⊕ [基于 Python 3 新增的函数注解（Function Annotations ）语法实现参数类型检查功能 - Huang Huang 的博客](https://mozillazg.com/2016/01/python-function-argument-type-check-base-on-function-annotations.html)


In [5]:
def foobar(a: int, b: "it's b", c: str = 5) -> tuple:
    return a, b, c
foobar.__annotations__


{'a': int, 'b': "it's b", 'c': str, 'return': tuple}

In [10]:
import inspect
sig = inspect.signature(foobar)
print(sig)

for k, v in sig.parameters.items():
    print('{k}: {a!r}'.format(k=k, a=v.annotation))     

print("return ->", sig.return_annotation)

(a:int, b:"it's b", c:str=5) -> tuple
a: <class 'int'>
b: "it's b"
c: <class 'str'>
return -> <class 'tuple'>


In [11]:
foobar(a='a', b=2, c=3)

('a', 2, 3)

- a: int 这种是注解参数
- c: str = 5 是注解有默认值的参数
- -> tuple 是注解返回值。

In [2]:
def foobar(a: 1+1) -> 2 * 2:
    return a

In [4]:
import collections
import functools
import inspect


def check(func):
    msg = ('Expected type {expected!r} for argument {argument}, '
           'but got type {got!r} with value {value!r}')
    # 获取函数定义的参数
    sig = inspect.signature(func)
    parameters = sig.parameters          # 参数有序字典
    arg_keys = tuple(parameters.keys())   # 参数名称

    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        CheckItem = collections.namedtuple('CheckItem', ('anno', 'arg_name', 'value'))
        check_list = []

        # collect args   *args 传入的参数以及对应的函数参数注解
        for i, value in enumerate(args):
            arg_name = arg_keys[i]
            anno = parameters[arg_name].annotation
            check_list.append(CheckItem(anno, arg_name, value))

        # collect kwargs  **kwargs 传入的参数以及对应的函数参数注解
        for arg_name, value in kwargs.items():
            anno = parameters[arg_name].annotation
            check_list.append(CheckItem(anno, arg_name, value))

        # check type
        for item in check_list:
            if not isinstance(item.value, item.anno):
                error = msg.format(expected=item.anno, argument=item.arg_name,
                                   got=type(item.value), value=item.value)
                raise TypeError(error)

        return func(*args, **kwargs)

    return wrapper

In [2]:
@check
def foobar(a: int, b: str, c: float = 3.2) -> tuple:
    return a, b, c

In [3]:
foobar(1, 'b')

(1, 'b', 3.2)

In [17]:
# foobar('a', 'b')
# TypeError: Expected type <class 'int'> for argument a, but got type <class 'str'> with value 'a'

In [18]:
age: int = 5
print(age)

5


In [19]:
from typing import List

def print_names(names: List[str]) -> None:
    for student in names:
        print(student)

In [20]:
from typing import List, Tuple


# Declare a point type annotation using a tuple of ints of [x, y]
Point = Tuple[int, int]


# Create a function designed to take in a list of Points
def print_points(points: List[Point]):
    for point in points:
        print("X:", point[0], "  Y:", point[1])

In [21]:
from typing import Optional

def try_to_print(some_num: Optional[int]):
    if some_num:
        print(some_num)
    else:
        print('Value was None!')

In [22]:
from typing import Union

def print_grade(grade: Union[int, str]):
    if isinstance(grade, str):
        print(grade + ' percent')
    else:
        print(str(grade) + '%')

⊕ [输入 - 支持类型提示 - Python 3.7.1文档](https://docs.python.org/3/library/typing.html)


In [24]:
from typing import NamedTuple
class Employee(NamedTuple):
    name: str
    id: int = 3

employee = Employee('Guido')
assert employee.id == 3